# Section 0

QPM: Assignment 3

DENG Yunyun <br>
LI Jiaxin <br>
SBAI Ilyas <br>
ZHOU Zhichen <br>

Note: NA

# Section 1

We retrieve a dataframe of the list of the S&P500 consistuents from Wikipedia and store it in ```tic```. We then replace any '.' in the tickers' names by a '-' to match the format required by yfinance. We finally turn this dataframe into a unique list of tickers that we use to retrive the stocks data using yfinance.

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

# Date range
start_date = "2000-01-01"
end_date = "2022-12-31"

# We retrieve the list of tickers of the S&P500 from Wikipedia
URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tic = pd.read_html(URL)[0]['Symbol']
tic = tic.str.replace('.', '-', regex=True) # using regex=True to suppress a warning related to a future change in pandas's feature
tic = tic.unique().tolist()

# Importing the stock data using the download function from the yf package
data = (yf.download(tickers=tic,
start=start_date,
end=end_date)
.reset_index()
.rename(columns = {"Date": "date",
                   "Open": "open",
"High": "high",
"Low": "low",
"Close": "close",
"Adj Close": "adjusted",
"Volume": "volume"
})
)
data.head()

[*********************100%%**********************]  503 of 503 completed


2 Failed downloads:
['VLTO', 'KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 946702800, endDate = 1672462800")


date   adjusted                                                  \
                      A AAL      AAPL ABBV ABNB       ABT      ACGL ACN   
0 2000-01-03  43.757759 NaN  0.847207  NaN  NaN  9.081108  1.277778 NaN   
1 2000-01-04  40.415146 NaN  0.775779  NaN  NaN  8.821647  1.270833 NaN   
2 2000-01-05  37.908180 NaN  0.787131  NaN  NaN  8.805433  1.388889 NaN   
3 2000-01-06  36.464790 NaN  0.719014  NaN  NaN  9.113542  1.375000 NaN   
4 2000-01-07  39.503510 NaN  0.753073  NaN  NaN  9.210837  1.451389 NaN   

              ... volume                                               \
        ADBE  ...   WYNN      XEL       XOM     XRAY XYL      YUM ZBH   
0  16.274670  ...    NaN  2738600  13458200   582300 NaN  3033493 NaN   
1  14.909397  ...    NaN   425200  14510800   317700 NaN  3315031 NaN   
2  15.204173  ...    NaN   500200  17485000  1188000 NaN  4642602 NaN   
3  15.328286  ...    NaN   344100  19461600   534300 NaN  3947658 NaN   
4  16.072981  ...    NaN   469500  16603800  1401000 NaN  6063647 NaN   

                         
      ZBRA     ZION ZTS  
0  1055700  1199600 NaN  
1   522450   816100 NaN  
2   612225  1124700 NaN  
3   263925  1112100 NaN  
4   333900   782000 NaN  

[5 rows x 3019 columns]

In [2]:
data = data.dropna(axis=1, how = 'all')
data = data.loc[:, (data.isna().sum(axis=0) <= 100)]
prices = data['adjusted']
prices.head()

,A,AAPL,ABT,ACGL,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WMT,WRB,WST,WY,XEL,XOM,XRAY,YUM,ZBRA,ZION
0,43.757759,0.847207,9.081108,1.277778,16.274670,28.706923,6.514026,25.075768,8.052906,10.858141,...,43.717712,1.238296,5.431208,11.691636,7.112457,18.668203,6.695474,4.725972,25.027779,36.146141
1,40.415146,0.775779,8.821647,1.270833,14.909397,27.254667,6.446173,25.075768,7.660815,10.858141,...,42.081882,1.200194,5.487315,11.252423,7.276230,18.310635,6.695474,4.630977,24.666668,34.395847
2,37.908180,0.787131,8.805433,1.388889,15.204173,27.652536,6.344393,24.834650,7.178246,11.278188,...,41.223061,1.181144,5.464870,11.848501,7.556990,19.308855,6.811221,4.654724,25.138889,34.355129
3,36.464790,0.719014,9.113542,1.375000,15.328286,26.896574,6.378315,25.166182,6.740915,11.236178,...,41.672913,1.249726,5.408764,12.402761,7.486797,20.307064,6.820125,4.615144,23.777779,34.843597
4,39.503510,0.753073,9.210837,1.451389,16.072981,27.652536,6.480096,25.738836,7.540174,11.404201,...,44.821899,1.230675,5.464870,11.994906,7.486797,20.247471,6.802316,4.512233,23.513889,34.925011


We save the data in a database for future reference.

In [40]:
import sqlite3

# We only keep the columns that have some values and we stack the data because the 
# column limit is 2000
#data = data.dropna(axis=1, how = 'all')
#data = data.stack()

#We put the price data in the right format
pricesql = prices.set_index(data['date'])

SnP500 = sqlite3.connect(
  database="/Users/ilyas/Library/CloudStorage/OneDrive-Personnel/EDHEC/M2/Quantitative Portfolio Management/snp500.sqlite"
)

SnP500.execute("VACUUM")
(pricesql
  .to_sql(name="SP500dailyprice", 
          con=SnP500, 
          if_exists="replace",
          index = True)
)

5787

# Section 2

We compute the log returns for all the stocks by taking the natural logarithm of the ratio between the last price and the first price.<br>

In [4]:
log_ret = pd.DataFrame(np.log(prices.iloc[-1]/prices.iloc[0])).T
log_ret

,A,AAPL,ABT,ACGL,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WMT,WRB,WST,WY,XEL,XOM,XRAY,YUM,ZBRA,ZION
0,1.224081,5.027233,2.472565,3.894514,3.029077,1.728748,2.640323,2.237545,3.14438,2.079681,...,1.165164,4.05113,3.766576,0.930021,2.262965,1.751913,1.547917,3.28566,2.326791,0.25866


# Section 3

We compute the returns for all stocks for each day as the percentage change from the previous day. <br>
For these returns, we then compute the mean and volatility that we annualise by multiplying respectively by 252 and sqrt(252). <br>
We finally compute the Sharpe ratio as the ratio between the annualised mean and the annualised volatility of returns.

In [5]:
ret = pd.DataFrame(prices.pct_change())
annual_ret = (pd.DataFrame(ret.mean()).T)*252
annual_vol = (pd.DataFrame(np.sqrt(ret.var())).T)*np.sqrt(252)
Sharpe_ratio = pd.DataFrame(annual_ret.sub(0.03).divide(annual_vol))
Sharpe_ratio


,A,AAPL,ABT,ACGL,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WMT,WRB,WST,WY,XEL,XOM,XRAY,YUM,ZBRA,ZION
0,0.262212,0.681875,0.440888,0.652109,0.452909,0.31547,0.430747,0.393565,0.465204,0.385733,...,0.205988,0.677416,0.606118,0.20132,0.395611,0.306854,0.273834,0.537104,0.377925,0.179681


# Section 4
It would **not** make sense to choose weights based only on the stocks' Sharpe ratio because this metric ignores the correlation between stocks. Another limitation of our computation of the Sharpe ratio is that we assumed returns to be IID when computing the annualised returns and volatility (by using the multipliers 252 and sqrt(252)), which is too strong of an assumption